In [1]:
import joblib
from joblib import Parallel, delayed
from multiprocessing import Queue
from itertools import product

from time import sleep

import subprocess

import os

import sys
sys.executable

'/trinity/home/r.zagidullin/miniconda3/envs/new_lama_venv/bin/python3.8'

In [2]:
# Define number of GPUs available
NAME = 'LAMA_PRESET_DP_SMALL_TO_OLD'
PREFIX = 'runs'
#GPU_PARTS = [[0, 1]]
gpu = [0,1]
RUNNER_PATH = 'runners/lama_dp.py'
CONFIG_PATH = 'runners/lama_dp.yml'
DEBUG = True

TIMEOUT = 7200
HARD_TIMEOUT = 7200
NTHREADS = 8
SEED = 42

# benchmark_params
data_path = '../../data/old_presets/data'
benchmark_path = '../../data/old_presets'

In [11]:
## Put indices in queue
#q = Queue(maxsize=len(GPU_PARTS))
#for i in GPU_PARTS:
#    q.put(i)

data_info = joblib.load(os.path.join(benchmark_path, 'data_info.pkl'))

In [3]:
def run_task(benchmark_path, data_path, dataset, fold, rewrite=False):
    gpu=[0,1]
    run_name = dataset + '_' + str(fold)
    
    print('Processing', run_name)
    
    benchmark_path = os.path.abspath(benchmark_path)
    data_path = os.path.abspath(data_path)
    output = os.path.join(benchmark_path, PREFIX, NAME, dataset, 'fold_{0}'.format(fold))
    
    os.makedirs(output, exist_ok=True)
    
    success_flg = os.path.join(output, 'SUCCESS')
    
    if os.path.exists(success_flg) and not rewrite:
        return 
    
    # clean folder
    for f in (x for x in os.listdir(output) if not x.startswith('.')):
        os.remove(os.path.join(output, f))
    
    # TRAIN
    try:
        
        script = "source /trinity/home/r.zagidullin/miniconda3/bin/activate; conda activate new_lama_venv; which python; "
        
        log = subprocess.check_output(script + ' '.join([
            'python', 
            RUNNER_PATH,
            '-b', benchmark_path,
            '-p', data_path,
            '-k', dataset,
            '-f', str(fold), 
            '-n', str(NTHREADS),
            '-s', str(SEED),
            '-d', ','.join(map(str, gpu)),
            '-c', os.path.abspath(CONFIG_PATH),
            '-t', str(TIMEOUT),
            '-o', output

        ]), timeout=HARD_TIMEOUT, shell=True, stderr=subprocess.STDOUT, executable='/bin/bash').decode()
        
        if DEBUG:
            print(log)
        
        with open(success_flg, 'w') as f:
            pass
        
        with open(os.path.join(output, 'train_log.txt'), 'w') as f:
            f.write(log)
    
    except subprocess.CalledProcessError as e:

        print(e.output.decode())
        
        with open(os.path.join(output, 'ERROR'), 'w') as f:
            pass
        
        with open(os.path.join(output, 'train_log.txt'), 'w') as f:
            f.write(e.output.decode())
                  
    except subprocess.TimeoutExpired:
        
        
        with open(os.path.join(output, 'TIMEOUT'), 'w') as f:
            pass
        
        print('HARD TIMEOUT!')    
    
# def run_task(benchmark_path, data_path, dataset, fold, rewrite=False):
    
#     gpu = q.get()
#     _run_task(benchmark_path, data_path, dataset, fold, gpu, rewrite)
#     q.put(gpu)
    
#     return 

In [12]:
data_info.keys()

dict_keys(['covertype', 'albert', 'higgs', 'guillermo', 'bank-marketing', 'numerai28.6', 'volkert', 'adult', 'MiniBooNE', 'dilbert', 'riccardo', 'shuttle', 'KDDCup09_appetency', 'Fashion-MNIST', 'connect-4', 'airlines', 'jannis', 'nomao', 'Amazon_employee_access', 'robert', 'aps_failure', 'jungle_chess_2pcs_raw_endgame_complete', 'ashrae-energy-prediction', 'ieee-fraud-detection', 'bnp-paribas-cardif-claims-management', 'porto-seguro-safe-driver-prediction', 'springleaf-marketing-response', 'talkingdata-adtracking-fraud-detection'])

In [6]:
data_info['springleaf-marketing-response']

{'path': 'springleaf-marketing-response/train.csv',
 'target': 'target',
 'task_type': 'binary',
 'drop': ['ID']}

In [17]:
!ls


catboost_info		  RunConfig_0_OLD.ipynb       runners
PrepareBenchmark.ipynb	  RunConfig_0_OLD_PF.ipynb
RunConfig_0_OLD_DP.ipynb  RunConfig_0_UTILIZED.ipynb


In [18]:
run_task(benchmark_path, data_path, 'shuttle', 4, True)

Processing shuttle_4
/trinity/home/r.zagidullin/miniconda3/envs/new_lama_venv/bin/python
Train dataset shuttle, fold 4
{'na_values': '?'}
   A1  A2  A3  A4  A5  A6  A7  A8  A9  class
0  50  21  77   0  28   0  27  48  22      1
1  55   0  92   0   0  26  36  92  56      3
2  53   0  82   0  52  -5  29  30   2      0
3  37   0  76   0  28  18  40  48   8      0
4  37   0  79   0  34 -26  43  46   2      0
/trinity/home/r.zagidullin/miniconda3/envs/new_lama_venv/lib/python3.8/site-packages/distributed/comm/ucx.py:61: UserWarning: A CUDA context for device 0 already exists on process ID 2295887. This is often the result of a CUDA-enabled library calling a CUDA runtime function before Dask-CUDA can spawn worker processes. Please make sure any such function calls don't happen at import time or in the global scope of a program.
  warnings.warn(
2022-11-17 14:53:36,233 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-vcw7cvj9', purging
2022-1

In [4]:
run_task(benchmark_path, data_path, 'robert', 4, True)

Processing robert_4


KeyboardInterrupt: 

In [11]:
data_info['robert']

{'path': 'openml/robert.csv',
 'target': 'class',
 'task_type': 'multiclass',
 'read_csv_params': {'na_values': '?'}}

In [ ]:
from time import sleep

In [ ]:
for d in data_info:
    run_task(benchmark_path, data_path, d, 4, True)
    sleep(20)

In [ ]:

def get_errs(res):
    
    errs = []
    path = os.path.join(benchmark_path, 'runs', res)
    files = os.listdir(path)

    for dat in os.listdir(path):
        fpath = os.path.join(path, dat)
        for f in os.listdir(fpath):
            if os.path.exists(os.path.join(fpath, f, 'ERROR')):
                errs.append((dat, f))
    
    return errs
    

In [ ]:
NAME

In [ ]:
get_errs(NAME)

In [ ]:
get_errs('LAMA_PRESET_SINGLE_GPU_SMALL_TO_OLD')

In [ ]:
import cupy
cupy.__version__